# Setup

In [1]:
from piper import piper
from piper.verbs import *
from piper.factory import get_sample_data

piper version 0.0.7, last run: Friday, 26 February 2021 14:49:52


# Sample data

In [2]:
df = get_sample_data()
head(df, 10)

367 rows, 7 columns


,dates,order_dates,countries,regions,ids,values_1,values_2
0,2020-01-01,2020-01-07,Italy,East,A,311,26
1,2020-01-02,2020-01-08,Portugal,South,D,150,375
2,2020-01-03,2020-01-09,Spain,East,A,396,88
3,2020-01-04,2020-01-10,Italy,East,B,319,233
4,2020-01-05,2020-01-11,Italy,East,D,261,187
5,2020-01-06,2020-01-12,Switzerland,North,D,155,253
6,2020-01-07,2020-01-13,Portugal,South,D,141,44
7,2020-01-08,2020-01-14,Italy,South,E,273,56
8,2020-01-09,2020-01-15,Italy,East,B,385,92
9,2020-01-10,2020-01-16,Sweden,North,C,331,148


# Examples

## groupby/add_group_calc

In [3]:
%%piper

df.groupby(['countries', 'regions']).agg(TotSales1=('values_1', 'sum'))
>> add_group_calc(index=['countries'], value='TotSales1',
                  sort_values=['countries', 'group_%'], 
                  ascending=[True, False])
>> head(8)
>> adorn()

32 rows, 2 columns


TotSales1  group_%
countries regions                    
France    West          4861    42.55
          North         2275    19.91
          East          2170    19.00
          South         2118    18.54
Germany   North         2239    30.54
          East          1764    24.06
          South         1753    23.91
          West          1575    21.48
          All          18755   199.99

## count with add_total option

In [4]:
count(df, 'countries', add_total=True)

,index,n
0,France,56
1,Spain,47
2,Italy,47
3,Norway,46
4,Switzerland,45
5,Sweden,45
6,Portugal,43
7,Germany,38
8,Total,367


## counts() - multindex total with adorn()

In [5]:
%%piper
counts(df, ['regions', 'countries'], sort_values=None)
>> pd.DataFrame.sort_values(['regions', 'n'], ascending=[True, False])
>> pd.DataFrame.reset_index()
>> head(4)
>> adorn(ignore_row_index=True)

32 rows, 3 columns


,regions,countries,n
0,East,Norway,20
1,East,Italy,17
2,East,France,14
3,East,Portugal,12
4,All,NaN,63


## counts() - multindex total with adorn()

In [6]:
%%piper

df.groupby(['countries', 'regions']).agg(total=('values_1', 'sum'))
>> where("countries.isin(['France', 'Italy', 'Germany'])")
>> add_group_calc(index='countries')
>> adorn(axis='row', ignore_row_index=False)
>> adorn(axis='column', columns='total')

total  group_%    All
countries regions                       
France    East      2170    19.00   2170
          North     2275    19.91   2275
          South     2118    18.54   2118
          West      4861    42.55   4861
Germany   East      1764    24.06   1764
          North     2239    30.54   2239
          South     1753    23.91   1753
          West      1575    21.48   1575
Italy     East      3023    30.54   3023
          North     1868    18.87   1868
          South     2520    25.45   2520
          West      2489    25.14   2489
          All      28655   299.99  28655

In [7]:
rule = 'A'
grouper = pd.Grouper(key='dates', freq=rule)
index=['regions', grouper]

gx = resample_pivot(df, index=index, grouper=grouper,
                    rule=rule, values=['values_1', 'values_2'])
gx = gx.unstack().fillna(0).astype(int)
gx = flatten_cols(gx, remove_prefix='val...')
adorn(gx, axis='both')

,1_2020,1_2021,2_2020,2_2021,All
East,19438,0,22349,0,41787
North,17205,372,17858,103,35538
South,17792,0,17869,0,35661
West,18797,0,16984,0,35781
All,73232,372,75060,103,148767


In [8]:
rule = 'A'
grouper = pd.Grouper(key='dates', freq=rule)
index=['regions', grouper]

In [9]:
%%piper
df 
>> resample_pivot(index=index,
                  grouper=grouper,
                  rule=rule, 
                  values=['values_1', 'values_2'])
>> adorn(axis='column', columns='values_1')
>> adorn(axis='row')

values_1  values_2    All
regions dates                           
East    2020      19438     22349  19438
North   2020      17205     17858  17205
        2021        372       103    372
South   2020      17792     17869  17792
West    2020      18797     16984  18797
        All       73604     75163  73604